In [54]:
import igl # Library to load meshes and perform operations on them
import meshplot as mp # Library to visualize meshes and point clouds
import vedo as vd # Library to visualize meshes and point clouds
import polyscope as ps # Library to visualize meshes
import numpy as np # Library to perform operations on matrices
import os # Library to perform operations on files and directories
import matplotlib.pyplot as plt

# Importing the classes and functions from the visualization folder
vd.settings.default_backend = 'k3d'

# Directory path
dir_path = os.getcwd()

In [109]:
v, f = igl.read_triangle_mesh('C:/Users/aikyna/Desktop/hananLab/hJupyter/models/Alisher/Extra/New_Tri_mesh.obj')
V = len(v) # Number of vertices
F = len(f) # Number of faces

v1, v2, k1, k2 = igl.principal_curvature(v, f)
normals = -igl.per_vertex_normals(v, f)

In [110]:
p = mp.plot(v, f, return_plot=True);
p.add_lines(v, v + normals, shading={"line_color": "red"});


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-7.037641…

In [111]:
vector = np.linalg.lstsq(normals, np.ones(V), rcond=None)[0]
vector = vector/np.linalg.norm(vector)
print(vector)

[ 0.2992187  -0.10266856  0.948645  ]


In [112]:
J = np.zeros((V + 1, 3))
r = np.zeros(V + 1)
X_0 = normals[0]
vec_length = 1

In [113]:
def compute_J(V, normals, X, vec_length):
    vec_var = X[0:3]
    for v_ind in range(V):
        J[v_ind, 0:3] = normals[v_ind] / vec_length
        
        r[v_ind] = np.dot(vec_var, normals[v_ind]) / vec_length - 1
    J[V, 0:3] = 2 * vec_var
    r[V] = np.dot(vec_var, vec_var) - 1

    vec_length = np.linalg.norm(vec_length)


In [114]:
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix, linalg

X = X_0

for i in range(20):
    compute_J(V, normals, X, vec_length)    

    H = J.T@J

    H[np.diag_indices_from(H)] += np.diag(H).max()*1e-6

    # Sparse matrix H
    H = csc_matrix(H)

    # Solve for dx
    dx = linalg.spsolve(H, -J.T@r)
    # Update vertices
    X = X + 0.7*dx
    
    # energy
    energy = r.T@r
    print(f"energy: {energy}\t dx: {np.linalg.norm(dx)}")
print(X, np.linalg.norm(X))
print(vector)


energy: 9.013194174490673	 dx: 0.323019952347878
energy: 1.2047581124931157	 dx: 0.09797194998429609
energy: 0.5046631117773469	 dx: 0.029170317959426966
energy: 0.442243941714458	 dx: 0.008668997624729334
energy: 0.43667504606606367	 dx: 0.002575824619996964
energy: 0.43617796586744667	 dx: 0.0007653555804840922
energy: 0.43613358654409023	 dx: 0.00022741600676692746
energy: 0.43612962384063036	 dx: 6.757588116073939e-05
energy: 0.4361292699685111	 dx: 2.0080555248947268e-05
energy: 0.43612923836459583	 dx: 5.96723344314301e-06
energy: 0.436129235541837	 dx: 1.773305440191792e-06
energy: 0.4361292352896958	 dx: 5.26995790224149e-07
energy: 0.43612923526717134	 dx: 1.5661873064989623e-07
energy: 0.4361292352651589	 dx: 4.6547149107450236e-08
energy: 0.43612923526497926	 dx: 1.3834236380121143e-08
energy: 0.4361292352649632	 dx: 4.111780397511103e-09
energy: 0.43612923526496183	 dx: 1.2221292647685347e-09
energy: 0.4361292352649616	 dx: 3.63259334059414e-10
energy: 0.4361292352649614	 d

In [115]:
from math import pi

sum1 = 0
sum2 = 0
X = X / np.linalg.norm(X)
for v_ind in range(V):
    sum1 += np.arccos(np.dot(vector, normals[v_ind])) ** 2
    sum2 += np.arccos(np.dot(X, normals[v_ind])) ** 2
print(sum1, sum2)
print(np.linalg.norm(vector - X))

48.74390923510775 48.7154886902244
0.00048560637633815365


In [116]:
p = mp.plot(v, f, return_plot=True);
p.add_lines(v, v + normals, shading={"line_color": "red"});
p.add_lines(v, v + 10 * vector, shading={"line_color": "green"});

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-7.037641…